### Random forest

X=df[['productGroup','quantity','price','rrp','voucherAmount','deviceID']].fillna(0).values

```
array([[500693, 616188,    258,     12,      4,      1],
       [302990, 900570,    435,      7,      1,      2],
       [  1306,   1494,    937,     11,      1,      0],
       [    97,      7,      8,     79,      1,      0],
       [    14,      3,      0,      0,     18,      0],
       [    17,      1,      0,      0,      1,      9]])
       
error=924799

CV=([ 0.59158359,  0.58674417,  0.58654827])
```

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cross_validation import train_test_split
from datetime import datetime
import sklearn.cluster
from sklearn.decomposition import PCA
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from __future__ import division

In [64]:
def dmc_error(returnQuantity, prediction):
    #∑ |returnQuantity𝑖 − prediction𝑖 |
    return np.sum(np.abs( np.subtract(returnQuantity, prediction) ))

In [6]:
df=pd.read_csv('orders_train.txt', sep=';'); df

,orderID,orderDate,articleID,colorCode,sizeCode,productGroup,quantity,price,rrp,voucherID,voucherAmount,customerID,deviceID,paymentMethod,returnQuantity
0,a1000001,2014-01-01,i1000382,1972,44,3,1,10.00,29.99,0,0,c1010575,2,BPRG,0
1,a1000001,2014-01-01,i1000550,3854,44,3,1,20.00,39.99,0,0,c1010575,2,BPRG,0
2,a1000002,2014-01-01,i1001991,2974,38,8,1,35.00,49.99,0,0,c1045905,4,BPRG,0
3,a1000002,2014-01-01,i1001999,1992,38,8,1,49.99,49.99,0,0,c1045905,4,BPRG,1
4,a1000003,2014-01-01,i1001942,1968,42,8,1,10.00,35.99,0,0,c1089295,2,PAYPALVC,0
5,a1000003,2014-01-01,i1001942,1972,42,8,1,10.00,35.99,0,0,c1089295,2,PAYPALVC,0
6,a1000003,2014-01-01,i1001974,1001,42,8,1,25.00,39.99,0,0,c1089295,2,PAYPALVC,0
7,a1000003,2014-01-01,i1001976,3976,44,8,1,15.00,39.99,0,0,c1089295,2,PAYPALVC,0
8,a1000003,2014-01-01,i1002392,2493,42,13,0,0.00,59.99,0,0,c1089295,2,PAYPALVC,0
9,a1000004,2014-01-01,i1002457,1001,42,14,1,89.99,89.99,0,0,c1050116,3,BPRG,1


In [46]:
# FIXME: Sementara nan aku replace seadanya dulu. benerin aja
X=df[['productGroup','quantity','price','rrp','voucherAmount','deviceID']].fillna(0).values
y=df.returnQuantity

### TODO:

- 'colorCode', 'sizeCode' ada huruf I
- split orderDate
- kategori lewatin LabelEncoder apa ga usah?
- dan lain-lain https://docs.google.com/document/d/1TIXlB9brKg4FER7-6mkQoc07QKGlUtJ4u3UPnYS7g48/edit?ts=57052864

In [19]:
rfc=RandomForestClassifier()

In [47]:
cv_scores=cross_val_score(rfc, X, y); cv_scores

array([ 0.59158359,  0.58674417,  0.58654827])

In [74]:
rfc.fit(X,y)
y_pred=rfc.predict(X)

In [75]:
dmc_error(df.returnQuantity.values, np.array(y_pred))

924799

In [76]:
confusion_matrix(df.returnQuantity.values, np.array(y_pred))

array([[500693, 616188,    258,     12,      4,      1],
       [302990, 900570,    435,      7,      1,      2],
       [  1306,   1494,    937,     11,      1,      0],
       [    97,      7,      8,     79,      1,      0],
       [    14,      3,      0,      0,     18,      0],
       [    17,      1,      0,      0,      1,      9]])

In [ ]:
label='returnQuantity'
sns.pairplot(df, hue=label)

In [ ]:
rfr=RandomForestRegressor()
rfr.fit(X,y)
y_pred=rfr.predict(X)

#cara CV regresi?
